## Joint Shock via EWMA‐Covariance + Eigenvector

We estimated the most recent conditional covariance of our Δ-yields using an EWMA (λ=0.94) and extracted the principal eigenvector to define a coherent 1-in-200-year shock. The resulting Δ-rates are:

| Tenor | EWMA Up (Δ)    | EWMA Down (Δ)   |
|:-----:|--------------:|---------------:|
| **3M**   | –0.000265 (–2.65 bp) | +0.000265 (+2.65 bp) |
| **6M**   | –0.000213 (–2.13 bp) | +0.000213 (+2.13 bp) |
| **1Y**   | –0.000298 (–2.98 bp) | +0.000298 (+2.98 bp) |
| **3Y**   | –0.002440 (–24.40 bp) | +0.002440 (+24.40 bp) |
| **5Y**   | –0.002763 (–27.63 bp) | +0.002763 (+27.63 bp) |
| **10Y**  | –0.002755 (–27.55 bp) | +0.002755 (+27.55 bp) |
| **15Y**  | –0.002597 (–25.97 bp) | +0.002597 (+25.97 bp) |

**Interpretation:**  
- The **belly** of the curve (3Y–10Y) has the largest shocks (~25–28 bp), consistent with medium-tenor volatility peaks.  
- **Short rates** (3M–1Y) show smaller shocks (~2–3 bp), reflecting tighter clustering.  
- **Long rates** (15Y) lie in between (~26 bp), capturing level moves.  

These Δ-rates define our up- and down-shock curves. Next, we’ll apply them to today’s zero curve to price the bond portfolio.



In [ ]:

# 1) Build EWMA covariance of df_diff
λ = 0.94 # This is from the JPM paper, 1996 and s common choice in financial times series
rets = df_diff.values  # shape (T, N)
T, N = rets.shape

# weights: w_t = (1-λ)*λ^(T-1-t) normalized so sum=1
weights = (1-λ) * λ ** np.arange(T-1, -1, -1)
weights /= weights.sum()

# compute EWMA covariance: H = X' W X
# center rets at zero (they're already zero‐mean Δ‐yields)
H_ewma = (rets * weights[:, None]).T @ rets  # shape (N,N)

# 2) Eigen‐decompose
vals, vecs = np.linalg.eigh(H_ewma)
idx = np.argmax(vals)
λ_max = vals[idx]
e_max = vecs[:, idx]

# 3) 1-in-200 quantile constant
c = stats.chi2.ppf(0.995, df=N)

# 4) Shock vectors
shock_mag   = np.sqrt(c * λ_max)
delta_up    =  shock_mag * e_max
delta_down  = -shock_mag * e_max

# Package into a DataFrame
shocks_ewma = pd.DataFrame({
    "ewma_up":   delta_up,
    "ewma_down": delta_down
}, index=df_diff.columns).round(6)

print("EWMA‐based joint shocks (Δ yields):")
print(shocks_ewma)


In [ ]:
# 5. Bond Portfolio Valuation


# 5.2 Build stressed zero curves using EWMA shocks
base_rates = df_dec.iloc[-1].values
zc_up   = ZeroCurve(tenors, base_rates + shocks_ewma["ewma_up"].values)
zc_down = ZeroCurve(tenors, base_rates + shocks_ewma["ewma_down"].values)

# 5.3 Fixed‐rate bond class (from earlier)
class FixedRateBond:
    def __init__(self, issue, maturity, coupon, freq=2, par=1000000):
        self.issue, self.maturity = pd.to_datetime(issue), pd.to_datetime(maturity)
        self.coupon, self.freq, self.par = coupon, freq, par
    def cashflows(self):
        step = int(12/self.freq)
        dates = pd.date_range(self.issue + pd.DateOffset(months=step),
                              self.maturity, freq=f"{step}ME")
        cfs = [self.par*self.coupon/self.freq]*len(dates)
        cfs[-1] += self.par
        return list(zip(dates, cfs))
    def pv(self, curve, today):
        return sum(amt * curve.discount_factor((dt-today).days/365)
                   for dt, amt in self.cashflows())

# 5.4 Define portfolio
today = df_dec.index[-1]
bonds = [
    FixedRateBond(today, today + pd.DateOffset(years=2), 0.02),
    FixedRateBond(today, today + pd.DateOffset(years=5), 0.035),
    FixedRateBond(today, today + pd.DateOffset(years=10), 0.03),
]

# 5.5 PV under Base, Up, Down
results = []
for name, zc in [("Base", zc_base), ("Up", zc_up), ("Down", zc_down)]:
    pvs = [b.pv(zc, today) for b in bonds]
    results.append([name] + pvs + [sum(pvs)])
cols = ["Scenario", "Bond1", "Bond2", "Bond3", "Portfolio PV"]
pv_df = pd.DataFrame(results, columns=cols).round(2)
print(pv_df)
